In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from pathlib import Path

import torch
from geometric_vector_perceptron import GVP_Network

In [3]:
data_path = Path("../data/synthetic")

In [4]:
cnn = torch.from_numpy(np.load(data_path/"cnn.npy"))
synthetic = torch.from_numpy(np.load(data_path/"synthetic.npy"))
with np.load(data_path/"answers.npz") as data:
    off_center = torch.from_numpy(data["off_center"])
    perimeter = torch.from_numpy(data["perimeter"])

In [6]:
synthetic.shape

torch.Size([20000, 2, 100, 3])